In [79]:
]add BenchmarkTools

    Updating registry at `C:\Users\Utilisateur\.julia\registries\General.toml`
   Resolving package versions...
    Updating `C:\Users\Utilisateur\.julia\environments\v1.8\Project.toml`
  [6e4b80f9] + BenchmarkTools v1.3.2
  No Changes to `C:\Users\Utilisateur\.julia\environments\v1.8\Manifest.toml`


In [82]:
using LinearAlgebra
using Test
using BenchmarkTools

In [43]:
# Calculate the QR factorization of a bloc-diagonale matrix A_vect. It return a vector F containing Q_k and R_k matrix
# of each block.
function blocdiagQR(A_vect)
    Q = []
    R = []
    m_vect = []
    n_vect = []      
    
    for k = 1:size(A,1)
        push!(m_vect,size(A_vect[k],1))
        push!(n_vect,size(A_vect[k],2))
        Q1, R1 = qr(A_vect[k])
        push!(Q, Q1)
        push!(R, R1)
    end
    return Q, R
end

blocdiagQR (generic function with 1 method)

In [71]:
A = rand(4,4)
Q, R = qr(A)
display(Q)

4×4 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}:
 -0.957713     0.0266962   0.212594  -0.192033
 -0.00453621  -0.696006   -0.443266  -0.564863
 -0.283808     0.039914   -0.775911   0.561981
 -0.0470919   -0.716428    0.395325   0.572915

In [63]:
# Create a block-diag A_vect random from a nb of submatrix and maximum of column and row of each submatrix.
function CreateDiagBlock(nb_Matrix, m_max, n_max)
    A_vect = []
    m_vect = rand(1:m_max, nb_Matrix)
    n_vect = rand(1:n_max, nb_Matrix)
    for k = 1:nb_Matrix
        if m_vect[k]<n_vect[k]
            (m_vect[k],n_vect[k]) = (n_vect[k],m_vect[k])
        end
        push!(A_vect, rand(m_vect[k],n_vect[k]))
    end
    m = sum(m_vect)
    n = sum(n_vect)
    return A_vect, m , n 
end

# Create a block-diag A random from a nb of submatrix and maximum of column and row of each submatrix.
function CreateDiagBlockFull(nb_Matrix, m_max, n_max)
    m_vect = rand(1:m_max, nb_Matrix)
    n_vect = rand(1:n_max, nb_Matrix)
    for k = 1:nb_Matrix
        if m_vect[k]<n_vect[k]
            (m_vect[k],n_vect[k]) = (n_vect[k],m_vect[k])
        end 
    end
    A_full = zeros(sum(m_vect), sum(n_vect))
            
    for k = 1:nb_Matrix
        if k == 1
            A_full[1:m_vect[k],1:n_vect[k]] = rand(m_vect[k],n_vect[k])
        else
            A_full[sum(m_vect[1:k-1])+1:sum(m_vect[1:k]),sum(n_vect[1:k-1])+1:sum(n_vect[1:k])] = rand(m_vect[k],n_vect[k])
        end
    end
    m = sum(m_vect)
    n = sum(n_vect)
        
    return A_full, m, n
end

# Transform a block-diag A_vect to a block-diag A full
function VectToFull(A_vect)
    nb_Matrix = size(A_vect,1)
    m_vect = []
    n_vect = []
    for k = 1:nb_Matrix
        push!(m_vect,size(A_vect[k],1))
        push!(n_vect,size(A_vect[k],2))
    end
    A_full = zeros(sum(m_vect), sum(n_vect))
    
    for k = 1:nb_Matrix
        if k == 1
            A_full[1:m_vect[k],1:n_vect[k]] = A_vect[k]
        else
            A_full[sum(m_vect[1:k-1])+1:sum(m_vect[1:k]),sum(n_vect[1:k-1])+1:sum(n_vect[1:k])] = A_vect[k]
        end
    end
    m = sum(m_vect)
    n = sum(n_vect)
    return A_full, m, n
end

VectToFull (generic function with 1 method)

In [60]:
# Test
A_vect = CreateDiagBlock(1, 10, 10)
F = blocdiagQR(A_vect)
A_full = CreateDiagBlockFull(4, 10, 10)

36×18 Matrix{Float64}:
 0.904841   0.0        0.0       0.0       …  0.0       0.0       0.0
 0.981571   0.0        0.0       0.0          0.0       0.0       0.0
 0.767117   0.0        0.0       0.0          0.0       0.0       0.0
 0.182768   0.0        0.0       0.0          0.0       0.0       0.0
 0.426983   0.0        0.0       0.0          0.0       0.0       0.0
 0.67282    0.0        0.0       0.0       …  0.0       0.0       0.0
 0.0895693  0.0        0.0       0.0          0.0       0.0       0.0
 0.884353   0.0        0.0       0.0          0.0       0.0       0.0
 0.314296   0.0        0.0       0.0          0.0       0.0       0.0
 0.520547   0.0        0.0       0.0          0.0       0.0       0.0
 0.0        0.379239   0.811545  0.213698  …  0.0       0.0       0.0
 0.0        0.0125879  0.214344  0.422886     0.0       0.0       0.0
 0.0        0.678795   0.080505  0.228257     0.0       0.0       0.0
 ⋮                                         ⋱  ⋮                   


In [75]:
# Tests

@testset begin
    A_vect, m, n = CreateDiagBlock(4, 10, 10)
    Q_vect, R_vect = blocdiagQR(A_vect)
    A_full, m1, n1 = VectToFull(A_vect)
    @test m1 == m && n1 == n
    b = rand(m)

    F_full = qr(A_full)
    
    Q_vect = VectToFull(Q_vect)
    @test norm(Q_vect'Q_vect - I) <= 1e-14 #tests that Q_H si unitary
    @test norm((Q_vect)*b - mult_Q_transpose_x!(R_vect,b)) <= 1e-14 #tests if the multiplication is correct
    @test norm((Q_vect')*b - mult_Q_x!(R_vect,b)) <= 1e-14
    @test norm(F_full.Q - Q_vect) <= 1e-14 #tests of unicity of QR decomposition
    @test norm(F_full.R - triu(R_vect[1:n,1:n])) <= 1e-14 
    @test norm(Q_vect*triu(R_vect) - A_full) <= 1e-14 #tests if the QR decomposition is correct
end


test set: Error During Test at In[75]:3
  Got exception outside of a @test
  MethodError: no method matching getindex(::LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}, Matrix{Float64}}, ::Int64)
  Stacktrace:
    [1] VectToFull(A_vect::LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}, Matrix{Float64}})
      @ Main .\In[63]:45
    [2] macro expansion
      @ In[75]:12 [inlined]
    [3] macro expansion
      @ C:\Users\Utilisateur\AppData\Local\Programs\Julia-1.8.5\share\julia\stdlib\v1.8\Test\src\Test.jl:1363 [inlined]
    [4] top-level scope
      @ In[75]:4
    [5] eval
      @ .\boot.jl:368 [inlined]
    [6] include_string(mapexpr::typeof(REPL.softscope), mod::Module, code::String, filename::String)
      @ Base .\loading.jl:1428
    [7] softscope_include_string(m::Module, code::String, filename::String)
      @ SoftGlobalScope C:\Users\Utilisateur\.julia\packages\SoftGlobalScope\u4UzH\src\SoftGlobalScope.jl:65
    [8] execute_request(socket::ZMQ.Socket, msg::IJulia.Msg)
     

LoadError: [91mSome tests did not pass: 1 passed, 0 failed, 1 errored, 0 broken.[39m

In [86]:
A_vect, m, n = CreateDiagBlock(10, 100, 100)
@benchmark blocdiagQR($A_vect)



BenchmarkTools.Trial: 4669 samples with 1 evaluation.
 Range (min … max):  884.400 μs …   7.028 ms  ┊ GC (min … max): 0.00% … 83.96%
 Time  (median):       1.019 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):     1.064 ms ± 342.203 μs  ┊ GC (mean ± σ):  2.30% ±  6.19%

   ▃▆▆█▆▁▆▃▇▃█▃▁▁                                                
  ▇██████████████▇▇▆▅▅▄▄▄▄▃▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂▂▂▁▂ ▄
  884 μs           Histogram: frequency by time         1.77 ms <

 Memory estimate: 307.34 KiB, allocs estimate: 56.

In [87]:
A_full, m1, n1 = VectToFull(A_vect)
@benchmark qr($A_full)

BenchmarkTools.Trial: 508 samples with 1 evaluation.
 Range (min … max):  8.241 ms … 18.787 ms  ┊ GC (min … max): 0.00% … 23.43%
 Time  (median):     9.566 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   9.830 ms ±  1.356 ms  ┊ GC (mean ± σ):  1.52% ±  5.42%

  ▆▃    ▂▅█▃▇▅▆▄▆▁▂ ▁                                         
  █████▇███████████▇█▆▆▅▄▇▃▄▃▃▃▂▃▃▃▂▃▂▃▂▂▁▂▁▂▁▂▁▁▁▂▂▂▃▃▁▃▁▂▂ ▄
  8.24 ms        Histogram: frequency by time        14.9 ms <

 Memory estimate: 1.75 MiB, allocs estimate: 6.